<span style="color:red"><b><<<<<<< local</b></span>

In [ ]:
"""
LenderChange is a class used from interbank.py to control the change of lender in a bank.
   It contains different
    - Boltzmann          using Boltzman probability to change
    - InitialStability   using a Barabási–Albert graph to initially assign relationships between banks
    - Preferential       using a Barabási-Albert with m edges for each new nde to set up only a set o links between
                            banks. In each step, new links between are set up based on the initial graph
    - RestrictedMarket   using an Erdos Renyi graph with p=parameter['p']. This method does not allow the evolution in
                            lender for each bank, it replicates the situation after a crisis when banks do not credit
    - ShockedMarket      using an Erdos Renyi graph with p=parameter['p']. In each step a new random Erdos Renyi is
                            used
    - ShockedMarket2     Erdos Renyi graph not directed and in each step, we randomly choose a direction
    - ShockedMarket3     Erdos Renyi graph with no limitation of links between nodes, and we randomly choose one of
                            it in each step.
    - SmallWorld         using a Watts and Strogatz algorithm with parameter['p'] and k=5
                            (Each node is joined with its k nearest neighbors in a ring topology)
@author: hector@bith.net
@date:   05/2023
"""
import random
import math
from sys import maxsize

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import sys
import inspect
import json
import warnings

In [ ]:
def determine_algorithm(given_name: str = "default"):
    default_method = "Boltzmann"
    if given_name.find('.')>0:
        given_name, given_parameter = given_name.split('.', maxsplit=1)
    else:
        given_parameter = None

    if given_name == "default":
        print(f"selected default method {default_method}")
        given_name = default_method
    if given_name == '?':
        for name, obj in inspect.getmembers(sys.modules[__name__]):
            if inspect.isclass(obj) and obj.__doc__:
                print("\t" + obj.__name__ + (" (default)" if name == default_method else '') + ':\n\t', obj.__doc__)
        sys.exit(0)
    else:
        for name, obj in inspect.getmembers(sys.modules[__name__]):
            if name.lower() == given_name.lower():
                if inspect.isclass(obj) and obj.__doc__:
                    lender_change_resultant =  obj()
                    if not given_parameter is None:
                        parameter,value = given_parameter.split('=')
                        lender_change_resultant.set_parameter(parameter, value)
                    return lender_change_resultant
        print(f"not found LenderChange algorithm with name '{given_name}'")
        sys.exit(-1)

In [ ]:
node_positions = None
node_colors = None

In [ ]:
def draw(original_graph, new_guru_look_for=False, title=None, show=False):
    """ Draws the graph using a spring layout that reuses the previous one layout, to show similar position for
        the same ids of nodes along time. If the graph is undirected (Barabasi) then no """
    graph_to_draw = original_graph.copy()
    plt.clf()
    if title:
        plt.title(title)
    global node_positions, node_colors
    # if not node_positions:
    guru = None
    if node_positions is None:
        node_positions = nx.spring_layout(graph_to_draw, pos=node_positions)
    if not hasattr(original_graph, "type") and original_graph.is_directed():
        # guru should not have out edges, and surely by random graphs it has:
        guru, _ = find_guru(graph_to_draw)
        for (i, j) in list(graph_to_draw.out_edges(guru)):
            graph_to_draw.remove_edge(i, j)
    if hasattr(original_graph, "type") and original_graph.type == "barabasi_albert":
        graph_to_draw, guru = get_graph_from_guru(graph_to_draw.to_undirected())
    if hasattr(original_graph, "type") and original_graph.type == "erdos_renyi" and graph_to_draw.is_directed():
        for node in list(graph_to_draw.nodes()):
            if not graph_to_draw.edges(node) and not graph_to_draw.in_edges(node):
                graph_to_draw.remove_node(node)
        new_guru_look_for = True
    if not node_colors or new_guru_look_for:
        node_colors = []
        guru, guru_node_edges = find_guru(graph_to_draw)
        for node in graph_to_draw.nodes():
            if node == guru:
                node_colors.append('darkorange')
            elif __len_edges(graph_to_draw, node) == 0:
                node_colors.append('lightblue')
            elif __len_edges(graph_to_draw, node) == 1:
                node_colors.append('steelblue')
            else:
                node_colors.append('royalblue')
    if hasattr(original_graph, "type") and original_graph.type == "barabasi_pref":
        nx.draw(graph_to_draw, pos=node_positions, node_color=node_colors, with_labels=True)
    else:
        nx.draw(graph_to_draw, pos=node_positions, node_color=node_colors, arrowstyle='->',
                arrows=True, with_labels=True)

    if show:
        plt.show()
    return guru

In [ ]:
def save_graph_png(graph, description, filename, add_info=False):
    if add_info:
        if not description:
            description = ""
        description += " " + GraphStatistics.describe(graph)

    guru = draw(graph, new_guru_look_for=True, title=description)
    plt.rcParams.update({'font.size': 6})
    plt.rcParams.update(plt.rcParamsDefault)
    warnings.filterwarnings("ignore", category=UserWarning)
    plt.savefig(filename)
    plt.close('all')
    return guru

In [ ]:
def save_graph_json(graph, filename):
    if graph:
        graph_json = nx.node_link_data(graph, edges="links")
        with open(filename, 'w') as f:
            json.dump(graph_json, f)

In [ ]:
def load_graph_json(filename):
    with open(filename, 'r') as f:
        graph_json = json.load(f)
        return nx.node_link_graph(graph_json, edges="links")

In [ ]:
def __len_edges(graph, node):
    if hasattr(graph, "in_edges"):
        return len(graph.in_edges(node))
    else:
        return len(graph.edges(node))

In [ ]:
def find_guru(graph):
    """It returns the guru ID and also a color_map with red for the guru, lightblue if weight<max/2 and blue others """
    guru_node = None
    guru_node_edges = 0
    for node in graph.nodes():
        edges_node = __len_edges(graph, node)
        if guru_node_edges < edges_node:
            guru_node_edges = edges_node
            guru_node = node
    return guru_node, guru_node_edges

In [ ]:
class WattsStrogatzGraph:
    def ring_lattice_edges(self, nodes, k):
        half = k // 2
        n = len(nodes)
        for i, v in enumerate(nodes):
            for j in range(i + 1, i + half + 1):
                w = nodes[j % n]
                yield v, w

    def make_ring_lattice(self, n, k):
        G = nx.DiGraph()
        nodes = range(n)
        G.add_nodes_from(nodes)
        edges = self.ring_lattice_edges(nodes, k)
        G.add_edges_from(edges)
        return G

    def rewire(self, G, p):
        for v, w in G.copy().edges():
            rewire= random.random()
            if rewire < p:
                G.remove_edge(v, w)
                choices = set(G) - {v} - set(G[v])
                new_w = random.choice(list(choices))
                G.add_edge(v, new_w)

    def new(self, n, p):
        G = self.make_ring_lattice(n=n, k=2)
        self.rewire(G, p)
        return G

In [ ]:
class GraphStatistics:
    @staticmethod
    def giant_component_size(graph):
        """weakly connected componentes of the directed graph using Tarjan's algorithm:
           https://en.wikipedia.org/wiki/Tarjan%27s_strongly_connected_components_algorithm"""
        if graph.is_directed():
            return nx.average_clustering(graph)
        else:
            return len(max(nx.connected_components(graph), key=len))

    @staticmethod
    def get_all_credit_channels(graph):
        return graph.number_of_edges()

    @staticmethod
    def avg_clustering_coef(graph):
        """clustering coefficient 0..1, 1 for totally connected graphs, and 0 for totally isolated
           if ~0 then a small world"""
        try:
            return nx.average_clustering(graph, count_zeros=True)
        except ZeroDivisionError:
            return 0

    @staticmethod
    def communities(graph):
        """Communities using greedy modularity maximization"""
        return list(nx.weakly_connected_components(graph if graph.is_directed() else graph.to_directed()))

    @staticmethod
    def grade_avg(graph):
        communities = GraphStatistics.communities(graph)
        total = 0
        for community in communities:
            total += len(community)
        return total / len(communities)

    @staticmethod
    def communities_not_alone(graph):
        """Number of communities that are not formed only with an isolated node"""
        total = 0
        for community in GraphStatistics.communities(graph):
            total += len(community) > 1
        return total

    @staticmethod
    def describe(graph, interact=False):
        if isinstance(graph, str):
            graph_name = graph
            try:
                graph = load_graph_json(graph_name)
            except FileNotFoundError:
                print("json file does not exist: %s" % graph)
                sys.exit(0)
            except (UnicodeDecodeError, json.decoder.JSONDecodeError) as e:
                print("json file does not contain a valid graph: %s" % graph)
                sys.exit(0)
        else:
            graph_name = '?'
        communities = GraphStatistics.communities(graph)
        string_result = f"giant={GraphStatistics.giant_component_size(graph)} " + \
                        f" comm_not_alone={GraphStatistics.communities_not_alone(graph)}" + \
                        f" comm={len(communities)}" + \
                        f" gcs={GraphStatistics.giant_component_size(graph)}"
        if interact:
            import code
            print(string_result)
            print(f"grade_avg={GraphStatistics.grade_avg(graph)}")
            print("communities=", list(GraphStatistics.communities(graph)))
            print(f"\n{graph_name} loaded into 'graph'\n")
            code.interact(local=locals())
            sys.exit(0)
        else:
            return string_result

In [ ]:
def __get_graph_from_guru(input_graph, output_graph, current_node, previous_node):
    """ It generates a new graph starting from the guru"""
    if __len_edges(input_graph, current_node) > 1:
        for (_, destination) in input_graph.edges(current_node):
            if destination != previous_node:
                __get_graph_from_guru(input_graph, output_graph, destination, current_node)
    if previous_node is not None:
        output_graph.add_edge(current_node, previous_node)

In [ ]:
def get_graph_from_guru(input_graph):
    guru, _ = find_guru(input_graph)
    output_graph = nx.DiGraph()
    __get_graph_from_guru(input_graph, output_graph, guru, None)
    return output_graph, guru

In [ ]:
def from_graph_to_array_banks(banks_graph, this_model):
    """ From the graph to a lender for each possible bank (or None if no links in the graph)"""
    if not banks_graph.is_directed():
        banks_graph = banks_graph.to_directed()
    for node in banks_graph:
        this_model.banks[node].lender = get_new_out_edge(banks_graph, node)

In [ ]:
def get_new_out_edge(banks_graph, node):
    if banks_graph.out_edges(node):
        edges = list(banks_graph.out_edges(node))
        edge_selected = random.randrange(len(edges))
        return edges[edge_selected][1]
    else:
        return None

In [ ]:
# ---------------------------------------------------------
# prototype
class LenderChange:
    GRAPH_NAME = ""

    def __init__(self):
        self.parameter = {}
        self.initial_graph_file = None

    def __str__(self):
        return "lc=LenderChangePrototype"

    def initialize_bank_relationships(self, this_model):
        """ Call once at initialize() model """
        pass

    def extra_relationships_change(self, this_model):
        pass

    def step_setup_links(self, this_model):
        """ Call at the end of each step """
        # if we remove banks from the array of banks, and there is no step_setup_links() code itself defined
        # in the LenderChange() class, we should at least call initialize_bank_relationships() to be sure
        # there are no links to removed banks:
        if not this_model.config.allow_replacement_of_bankrupted:
            self.initialize_bank_relationships(this_model)

    def change_lender(self, this_model, bank, t):
        """ Call at the end of each step, for each bank and before going to the next step"""
        pass

    def new_lender(self, this_model, bank):
        """ Describes the mechanism of change"""
        pass

    def set_initial_graph_file(self, lc_ini_graph_file):
        self.initial_graph_file = lc_ini_graph_file

    def describe(self):
        return ""

    def check_parameter(self, name, value):
        """ Called after set_parameter() to verify that the necessary parameters are set """
        return False

    def set_parameter(self, name, value):
        if not value is None:
            if self.check_parameter(name, value):
                self.parameter[name] = float(value)
            else:
                print(f"error with parameter '{name}' for {self.__class__.__name__}")
                sys.exit(-1)

    def get_credit_channels(self):
        if hasattr(self, "banks_graph"):
            return GraphStatistics.get_all_credit_channels(self.banks_graph)
        else:
            return None

---------------------------------------------------------

In [ ]:
class Boltzmann(LenderChange):
    """It chooses randomly a lender for each bank and in each step changes using Boltzmann's probability
         Also it has parameter γ to control the change [0..1], 0=Boltzmann only, 1 everyone will move randomly
    """
    # parameter to control the change of guru: 0 then Boltzmann only, 1 everyone will move randomly
    gamma: float = 0.5  # [0..1] gamma
    CHANGE_LENDER_IF_HIGHER = 0.5

    def __str__(self):
        return "lc=BoltzMann"

    def initialize_bank_relationships(self, this_model):
        if self.initial_graph_file:
            graph = load_graph_json(self.initial_graph_file)
            from_graph_to_array_banks(graph, this_model)
        if this_model.export_datafile:
            this_model.statistics.get_graph(0)

    def change_lender(self, this_model, bank, t):
        """ It uses γ but only after t=20, at the beginning only Boltzmann"""
        possible_lender = self.new_lender(this_model, bank)
        if possible_lender is None:
            possible_lender_mu = 0
        else:
            possible_lender_mu = this_model.banks[possible_lender].mu
        if bank.get_lender() is None:
            current_lender_mu = 0
        else:
            current_lender_mu = bank.get_lender().mu

        # we can now break old links and set up new lenders, using probability P
        # (equation 8)
        boltzmann = 1 / (1 + math.exp(-this_model.config.beta * (possible_lender_mu - current_lender_mu)))

        if t < 20:
            # bank.P = 0.35
            # bank.P = random.random()
            bank.P = boltzmann
            # option a) bank.P initially 0.35
            # option b) bank.P randomly
            # option c) with t<=20 boltzmann, and later, stabilize it
        else:
            bank.P_yesterday = bank.P
            # gamma is not sticky/loyalty, persistence of the previous attitude
            bank.P = self.gamma * bank.P_yesterday + (1 - self.gamma) * boltzmann

        if bank.P >= self.CHANGE_LENDER_IF_HIGHER:
            text_to_return = f"{bank.get_id()} new lender is #{possible_lender} from #{bank.lender} with %{bank.P:.3f}"
            bank.lender = possible_lender
        else:
            text_to_return = f"{bank.get_id()} maintains lender #{bank.lender} with %{1 - bank.P:.3f}"
        return text_to_return

    def new_lender(self, this_model, bank):
        """ It gives to the bank a random new lender. It's used initially and from change_lender() """
        # r_i0 is used the first time the bank is created:
        if bank.lender is None:
            bank.rij = np.full(this_model.config.N, this_model.config.r_i0, dtype=float)
            bank.rij[bank.id] = 0
            bank.mu = 0
            bank.asset_i = 0
            bank.asset_j = 0
            bank.asset_equity = 0
            # if it's just created, only not to be ourselves is enough
            new_value = random.randrange(this_model.config.N - 1)
        else:
            # if we have a previous lender, new should not be the same
            new_value = random.randrange(this_model.config.N - 2 if this_model.config.N > 2 else 1)

        if this_model.config.N == 2:
            new_value = 1 if bank.id == 0 else 0
        else:
            if new_value >= bank.id:
                new_value += 1
                if bank.lender is not None and new_value >= bank.lender:
                    new_value += 1
            else:
                if bank.lender is not None and new_value >= bank.lender:
                    new_value += 1
                    if new_value >= bank.id:
                        new_value += 1
        return new_value

    def describe(self):
        return f"($\\gamma={self.gamma} and change if >{self.CHANGE_LENDER_IF_HIGHER})$"

In [ ]:
class InitialStability(Boltzmann):
    """ We define a Barabási–Albert graph with 1 edges for each node, and we convert it to a directed graph.
          It is used for initially have more stable links between banks
    """

    CHANGE_LENDER_IF_HIGHER = 0.8
    GRAPH_NAME = 'barabasi_albert'

    def __str__(self):
        return "lc=InitialStability"

    def __create_directed_graph_from_barabasi_albert(self, barabasi_albert, result, current_node, previous_node):
        if len(barabasi_albert.edges(current_node)) > 1:
            for (_, destination) in barabasi_albert.edges(current_node):
                if destination != previous_node:
                    self.__create_directed_graph_from_barabasi_albert(barabasi_albert, result,
                                                                      destination, current_node)
                # if not previous_node:
                #    result.add_edge(destination, current_node)
        if previous_node is not None:
            result.add_edge(current_node, previous_node)

    def initialize_bank_relationships(self, this_model):
        if self.initial_graph_file:
            self.banks_graph = load_graph_json(self.initial_graph_file)
            description = f"{self.GRAPH_NAME} from file {self.initial_graph_file}"
        else:
            self.banks_graph, _ = get_graph_from_guru(nx.barabasi_albert_graph(this_model.config.N, 1))
            description = f"{self.GRAPH_NAME} m=1 {GraphStatistics.describe(self.banks_graph)}"
        self.banks_graph.type = self.GRAPH_NAME
        if this_model.export_datafile:
            save_graph_png(self.banks_graph, description,
                           this_model.statistics.get_export_path(this_model.export_datafile, f"_{self.GRAPH_NAME}.png"))
            save_graph_json(self.banks_graph,
                            this_model.statistics.get_export_path(this_model.export_datafile,
                                                                  f"_{self.GRAPH_NAME}.json"))
        from_graph_to_array_banks(self.banks_graph, this_model)
        return self.banks_graph

In [ ]:
class Preferential(Boltzmann):
    """ Using a Barabasi with grade m we restrict to those relations the possibilities to obtain an outgoing link
          (a lender). To improve the specialization of banks, granting 3*C0 to the guru, 2*C0 to its neighbours
          and C to the others. To balance those with 2C0 and 3C0, we will reduce D
        In each step, we change the lender using the base self.banks_graph_full
    """
    banks_graph = None
    guru = None
    GRAPH_NAME = "barabasi_pref"

    def __str__(self):
        if 'm' in self.parameter:
            return f"lc=Preferential.m={self.parameter['m']}"
        else:
            return f"lc=Preferential"

    def check_parameter(self, name, value):
        if name == 'm':
            try:
                value = int(value)
            except:
                pass
            if isinstance(value, int) and 1 <= value:
                return True
            else:
                print("value for 'm' should be an integer >= 1: %s" % value)
                return False
        else:
            return False

    def set_parameter(self, name, value):
        if not value is None:
            if self.check_parameter(name, value):
                self.parameter[name] = int(value)
            else:
                print(f"error with parameter '{name}' for {self.__class__.__name__}")
                sys.exit(-1)

    def initialize_bank_relationships(self, this_model):
        if self.initial_graph_file:
            self.banks_graph_full = load_graph_json(self.initial_graph_file)
            description = f"{self.GRAPH_NAME} from file {self.initial_graph_file}"
        else:
            self.banks_graph_full = nx.barabasi_albert_graph(this_model.config.N, self.parameter['m'])
            description = (f"{self.GRAPH_NAME} m={self.parameter['m']:5.3f} "
                           f"{GraphStatistics.describe(self.banks_graph_full)}")
        self.banks_graph_full.type = self.GRAPH_NAME
        if this_model.export_datafile:
            self.guru = save_graph_png(self.banks_graph_full, description,
                                       this_model.statistics.get_export_path(this_model.export_datafile,
                                                                             f"_{self.GRAPH_NAME}.png"))
        else:
            self.guru, _ = find_guru(self.banks_graph_full)
        self.full_barabasi_extract_random_directed(this_model)
        self.prize_for_good_banks(this_model)
        if this_model.export_datafile:
            save_graph_json(self.banks_graph,
                            this_model.statistics.get_export_path(this_model.export_datafile,
                                                                  f"_{self.GRAPH_NAME}.json"))
        return self.banks_graph

    def prize_for_good_banks(self, this_model):
        this_model.banks[self.guru].D -= this_model.banks[self.guru].C * 2
        this_model.banks[self.guru].C *= 3
        for (_, node) in self.banks_graph_full.edges(self.guru):
            this_model.banks[node].D -= this_model.banks[node].C
            this_model.banks[node].C *= 2

    def step_setup_links(self, this_model):
        self.full_barabasi_extract_random_directed(this_model)

    def full_barabasi_extract_random_directed(self, this_model, current_node=None):
        if current_node is None:
            self.banks_graph = nx.DiGraph()
            self.banks_graph.type = self.GRAPH_NAME
            current_node = self.guru
        edges = list(self.banks_graph_full.edges(current_node))
        self.banks_graph.add_node(current_node)
        for (_, destination) in edges[:]:
            if destination not in self.banks_graph.nodes():
                self.full_barabasi_extract_random_directed(this_model, destination)
        try:
            edges.remove((current_node, this_model.banks[current_node].lender))
        except ValueError:
            pass
        candidate = None
        while candidate is None and edges:
            candidate = random.choice(edges)[1]
            if (candidate, current_node) in self.banks_graph.edges():
                edges.remove((current_node, candidate))
                candidate = None
        if candidate is None:
            if this_model.banks[current_node].lender is not None:
                self.banks_graph.add_edge(current_node, this_model.banks[current_node].lender)
        else:
            self.banks_graph.add_edge(current_node, candidate)
        return current_node

    def new_lender(self, this_model, bank):
        if bank.lender is None:
            bank.rij = np.full(this_model.config.N, this_model.config.r_i0, dtype=float)
            bank.rij[bank.id] = 0
            bank.mu = 0
        if self.banks_graph and self.banks_graph.out_edges() and self.banks_graph.out_edges(bank.id):
            return list(self.banks_graph.out_edges(bank.id))[0][1]
        else:
            return None

    def describe(self):
        return f"($\\gamma={self.gamma} and change if >{self.CHANGE_LENDER_IF_HIGHER})$"

In [ ]:
class RestrictedMarket(LenderChange):
    """ Using an Erdos Renyi graph with p=parameter['p']. This method does not allow the evolution in
          lender for each bank, it replicates the situation after a crisis when banks do not credit
    """
    SAVE_THE_DIFFERENT_GRAPH_OF_EACH_STEP = True
    GRAPH_NAME = "erdos_renyi"

    def __str__(self):
        if 'p' in self.parameter:
            return f"lc=RestrictedMarket.p={self.parameter['p']}"
        else:
            return f"lc=RestrictedMarket"

    def generate_banks_graph(self, this_model):
        result = nx.DiGraph()
        result.add_nodes_from(list(range(this_model.config.N)))
        for i in range(this_model.config.N):
            if random.random() < self.parameter['p']:
                j = random.randrange(this_model.config.N)
                while j == i:
                    j = random.randrange(this_model.config.N)
                result.add_edge(i, j)
        return result, f"erdos_renyi p={self.parameter['p']:5.3} {GraphStatistics.describe(result)}"

    def check_parameter(self, name, value):
        if name == 'p':
            try:
                value = float(value)
            except:
                pass
            if isinstance(value, float) and 0 <= value <= 1:
                return True
            else:
                print("value for 'p' should be a float number >= 0 and <= 1: %s" % value)
                return False
        else:
            return False

    def initialize_bank_relationships(self, this_model, save_graph=True):
        """ It creates a Erdos Renyi graph with p defined in parameter['p']. No changes in relationships before end"""
        if self.initial_graph_file:
            self.banks_graph = load_graph_json(self.initial_graph_file)
            description = f"from file {self.initial_graph_file}"
        else:
            self.banks_graph, description = self.generate_banks_graph(this_model)
        self.banks_graph.type = self.GRAPH_NAME
        if this_model.export_datafile and save_graph:
            filename_for_file = f"_{self.GRAPH_NAME}"
            if self.SAVE_THE_DIFFERENT_GRAPH_OF_EACH_STEP:
                filename_for_file += f"_{this_model.t}"
            save_graph_png(self.banks_graph, description,
                         this_model.statistics.get_export_path(this_model.export_datafile, f"{filename_for_file}.png"))
            save_graph_json(self.banks_graph,
                         this_model.statistics.get_export_path(this_model.export_datafile, f"{filename_for_file}.json"))
        for (borrower, lender_for_borrower) in self.banks_graph.edges():
            this_model.banks[borrower].lender = lender_for_borrower
        return self.banks_graph

    def step_setup_links(self, this_model):
        # if not declared, at each step it will generate a new graph:
        pass

    def new_lender(self, this_model, bank):
        """ We return the same lender we have created in self.banks_graph """
        bank.rij = np.full(this_model.config.N, this_model.config.r_i0, dtype=float)
        bank.rij[bank.id] = 0
        #bank.r = this_model.config.r_i0
        bank.mu = 0
        bank.asset_i = 0
        bank.asset_j = 0
        bank.asset_equity = 0
        return bank.lender

    def change_lender(self, this_model, bank, t):
        """ We return the same lender we have created in self.banks_graph """
        bank.P = 0
        return f"{bank.get_id()} maintains lender #{bank.lender} with %1"

In [ ]:
class ShockedMarket(RestrictedMarket):
    """ Using an Erdos Renyi graph with p=parameter['p']. This method replicate RestrictedMarket
        but using a new network relationship between banks, but always with same p. So the links
        in t=i are destroyed and new aleatory links in t=i+1 are created using a new Erdos Renyi
        graph.
    """
    SAVE_THE_DIFFERENT_GRAPH_OF_EACH_STEP = False

    def __str__(self):
        if 'p' in self.parameter:
            return f"lc=ShockedMarket.p={self.parameter['p']}"
        else:
            return f"lc=ShockedMarket"


    def step_setup_links(self, this_model):
        """ At the end of each step, a new graph is generated """
        self.initialize_bank_relationships(this_model, save_graph=self.SAVE_THE_DIFFERENT_GRAPH_OF_EACH_STEP)

In [ ]:
class ShockedMarket2(ShockedMarket):
    """ Erdos Renyi graph not directed and in each step, we randomly choose a direction.
    """
    SAVE_THE_DIFFERENT_GRAPH_OF_EACH_STEP = False

    def __str__(self):
        if 'p' in self.parameter:
            return f"lc=ShockedMarket2.p={self.parameter['p']}"
        else:
            return f"lc=ShockedMarket2"

    def extra_relationships_change(self, this_model):
        for bank in this_model.banks:
            if bank.incrD < 0: # borrowers
                # we search all the possible lenders and we randomly choose one with incrD>0:
                possible_lenders = []
                for (_,j) in self.banks_graph.edges(bank.id):
                    if this_model.banks[j].incrD> 0:
                        possible_lenders.append(j)
                if possible_lenders:
                    bank.lender = random.choice(possible_lenders)
                else:
                    bank.lender = None

In [ ]:
class ShockedMarket3(ShockedMarket2):
    """ Erdos Renyi graph not directed and in each step with no limits in number of edges,
        and we randomly choose a direction.
    """
    SAVE_THE_DIFFERENT_GRAPH_OF_EACH_STEP = False

    def __str__(self):
        if 'p' in self.parameter:
            return f"lc=ShockedMarket3.p={self.parameter['p']}"
        else:
            return f"lc=ShockedMarket3"

    def generate_banks_graph(self, this_model):
        result = nx.erdos_renyi_graph(n=this_model.config.N, p=self.parameter['p'])
        return result, f"erdos_renyi p={self.parameter['p']:5.3} {GraphStatistics.describe(result)}"

    def extra_relationships_change(self, this_model):
        for bank in this_model.banks:
            if bank.incrD < 0: # borrowers
                # we search all the possible lenders and we randomly choose one with incrD>0:
                possible_lenders = []
                for (_,j) in self.banks_graph.edges(bank.id):
                    if this_model.banks[j].incrD> 0:
                        possible_lenders.append(j)
                if possible_lenders:
                    bank.lender = random.choice(possible_lenders)
                else:
                    bank.lender = None

In [ ]:
class SmallWorld(ShockedMarket):
    """ SmallWorld implementation using Watts Strogatz
    """
    GRAPH_NAME = "watts_strogatz"

    def __str__(self):
        if 'p' in self.parameter:
            return f"lc=SmallWorld.p={self.parameter['p']}"
        else:
            return f"lc=SmallWorld"


    def generate_banks_graph(self, this_model):
        generator = WattsStrogatzGraph()
        result = generator.new(n=this_model.config.N, p=self.parameter['p'])
        return result, f"watts_strogatz p={self.parameter['p']:5.3} {GraphStatistics.describe(result)}"

    def initialize_bank_relationships(self, this_model, save_graph=True):
        """ It creates a Erdos Renyi graph with p defined in parameter['p']. No changes in relationships before end"""
        if self.initial_graph_file:
            self.banks_graph = load_graph_json(self.initial_graph_file)
            description = f"from file {self.initial_graph_file}"
        else:
            self.banks_graph, description = self.generate_banks_graph(this_model)
        self.banks_graph.type = self.GRAPH_NAME
        if this_model.export_datafile and save_graph:
            save_graph_png(self.banks_graph, description,
                           this_model.statistics.get_export_path(this_model.export_datafile, f"_{self.GRAPH_NAME}.png"))
            save_graph_json(self.banks_graph,
                            this_model.statistics.get_export_path(this_model.export_datafile,
                                                                  f"_{self.GRAPH_NAME}.json"))
        for (borrower, lender_for_borrower) in self.banks_graph.edges():
            this_model.banks[borrower].lender = lender_for_borrower
        return self.banks_graph

    def new_lender(self, this_model, bank):
        """ Same lender we have created in self.banks_graph """
        bank.rij = np.full(this_model.config.N, this_model.config.r_i0, dtype=float)
        bank.rij[bank.id] = 0
        #bank.r = this_model.config.r_i0
        bank.mu = 0
        bank.asset_i = 0
        bank.asset_j = 0
        bank.asset_equity = 0
        return bank.lender

    def change_lender(self, this_model, bank, t):
        """ Lender is not changing in this model """
        bank.P = 0
        return f"{bank.get_id()} maintains lender #{bank.lender} with %1"

<span style="color:red"><b>=======</b></span>

In [ ]:
"""
Generates a simulation of an interbank network following the rules described in paper
  Reinforcement Learning Policy Recommendation for Interbank Network Stability
  from Gabrielle and Alessio

  You can use it interactively, but if you import it, the process will be:
    # model = Model()
    #   # step by step:
    #   model.enable_backward()
    #   model.forward() # t=0 -> t=1
    #   model.backward() : reverts the last step (when executed)
    #   # all in a loop:
    #   model.simulate_full()

@author: hector@bith.net
@date:   04/2023
"""
import copy
import random
import logging
import argparse
from typing import Any
import networkx as nx
import sys
import os
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import numpy as np
import lxml.etree
import lxml.builder
import gzip
import scipy.stats

In [ ]:
LENDER_CHANGE_DEFAULT = 'ShockedMarket3'
LENDER_CHANGE_DEFAULT_P = 0.333

In [ ]:
class Config:
    """
        Configuration parameters for the interbank network
    """
    T: int = 1000  # time (1000)
    N: int = 50    # number of banks (50)

    reserves: float = 0.02

    # seed applied for random values (set during initialize)
    seed : int = None

    # if False, when a bank fails it's not replaced and N is reduced
    allow_replacement_of_bankrupted = True

    # If true allow_replacement, then:
    #    - reintroduce=False we reintroduce bankrupted banks with initial values
    #    - reintroduce=True we reintroduce with median of current values
    reintroduce_with_median = False

    # if the then a gdt with all the data of time evolution of equity of each bank is generated:
    detailed_equity = False

    # shocks parameters: mi=0.7 omega=0.55 for perfect balance
    mu: float = 0.7  # mi µ
    omega: float = 0.55  # omega ω

    # Lender's change mechanism
    lender_change: LenderChange = None

    # screening costs
    phi: float = 0.025  # phi Φ
    chi: float = 0.015  # ji Χ

    xi: float = 0.3  # xi ξ liquidation cost of collateral
    rho: float = 0.3  # ro ρ fire sale cost

    beta: float = 5  # β beta intensity of breaking the connection (5)
    alfa: float = 0.1  # α alfa below this level of E or D, we will bankrupt the bank

    # If true, psi variable will be ignored:
    psi_endogenous = False
    psi: float = 0.3  # market power parameter : 0 perfect competence .. 1 monopoly

    # If it's a value greater than 0, instead of allowing interest rates for borrowers of any value, we normalize
    # them to range [r_i0..normalize_interest_rate_max]:
    normalize_interest_rate_max = 1

    # banks initial parameters
    # L + C + R = D + E
    # but R = 0.02*D and C_i0= 30-2.7=27.3 and R=2.7
    L_i0: float = 120  # long term assets
    C_i0: float = 30  # capital BEFORE RESERVES ESTIMATION, after it will be 27.3
    # R_i0=2.7
    D_i0: float = 135  # deposits
    E_i0: float = 15  # equity
    r_i0: float = 0.02  # initial rate

    # if enabled and != [] the values of t in the array (for instance [150,350]) will generate
    # a graph with the relations of the firms. If * all the instants will generate a graph, and also an animated gif
    # with the results
    GRAPHS_MOMENTS = []

    # what elements are in the results.csv file, and also which are plot.
    # 1 if also plot, 0 not to plot:
    ELEMENTS_STATISTICS = {'B': True, 'liquidity': True, 'interest_rate': True, 'asset_i': True, 'asset_j': True,
                           'equity': True, 'equity_borrowers': True, 'bankruptcy': True,
                           'potential_credit_channels': True,
                           'P': True, 'best_lender': True,
                           'policy': False, 'fitness': False, 'best_lender_clients': False,
                           'rationing': True, 'leverage': False, 'systemic_leverage': False,
                           'num_of_rationed': True,
                           'loans': False, 'c': True,
                           'reserves': True, 'deposits': True,
                           'psi': True, 'psi_lenders': True,
                           'potential_lenders':False,
                           'active_lenders': False, 'active_borrowers': False, 'prob_bankruptcy': False,
                           'num_banks': True, 'bankruptcy_rationed': True}


    def __str__(self, separator=''):
        description = sys.argv[0] if __name__ == '__main__' else ''
        for attr, value in self:
            description += ' {}={}{}'.format(attr, value, separator)
        return description + ' '

    def __iter__(self):
        for attr in dir(self):
            value = getattr(self, attr)
            if isinstance(value, int) or isinstance(value, float) or isinstance(value, bool):
                yield (attr, value)

    def get_current_value(self, name_config):
        current_value = None
        try:
            current_value = getattr(self, name_config)
        except AttributeError:
            logging.error("Config has no '{}' parameter".format(name_config))
            sys.exit(-1)
        # if current_value is None, then we cannot guess which type is the previous value:
        if current_value is None:
            try:
                current_value = self.__annotations__[name_config]
            except:
                return False
            if current_value is int:
                return 0
            elif current_value is bool:
                return False
            else:
                return 0.0
        return current_value


    def define_values_from_args(self, config_list):
        if config_list:
            config_list.sort()
            for item in config_list:
                if item == '?':
                    print(self.__str__(separator='\n'))
                    sys.exit(0)
                elif item.startswith('lc='):
                    # to define the lenderchange algorithm by command line:
                    self.lender_change = determine_algorithm(item.replace("lc=",''))
                else:
                    try:
                        name_config, value_config = item.split('=')
                    except ValueError:
                        name_config, value_config = ('-', '-')
                        logging.error('A Config value should be passed as parameter=value: {}'.format(item))
                        sys.exit(-1)
                    current_value = self.get_current_value(name_config)
                    try:
                        if isinstance(current_value, bool):
                            if value_config.lower() in ('y', 'yes', 't', 'true', 'on', '1'):
                                setattr(self, name_config, True)
                            elif value_config.lower() in ('n', 'no', 'false', 'f', 'off', '0'):
                                setattr(self, name_config, False)
                        elif isinstance(current_value, int):
                            setattr(self, name_config, int(value_config))
                        elif isinstance(current_value, float):
                            setattr(self, name_config, float(value_config))
                        else:
                            setattr(self, name_config, float(value_config))
                    except ValueError:
                        print(current_value, type(current_value), isinstance(current_value, bool),
                              isinstance(current_value, int))
                        logging.error('Value given for {} is not valid: {}'.format(name_config, value_config))
                        sys.exit(-1)

In [ ]:
class Statistics:
    lender_no_d = []
    no_lender = []
    enough_money = []
    bankruptcy = []
    best_lender = []
    best_lender_clients = []
    potential_credit_channels = []
    liquidity = []
    policy = []
    deposits = []
    reserves = []
    interest_rate = []
    incrementD = []
    fitness = []
    rationing = []
    leverage = []
    systemic_leverage = []
    loans = []
    asset_i = []
    asset_j = []
    equity = []
    equity_borrowers = []
    P = []
    B = []
    c = []
    P_max = []
    P_min = []
    P_std = []
    active_borrowers = []
    active_lenders = []
    potential_lenders = []
    prob_bankruptcy = []
    num_of_rationed = []
    psi = []
    psi_lenders = []
    num_banks = []
    bankruptcy_rationed = []
    model = None
    graphs = {}
    graphs_pos = None
    plot_format = None
    graph_format = '.svg'
    output_format = '.gdt'
    create_gif = False
    OUTPUT_DIRECTORY = 'output'
    NUMBER_OF_ITEMS_IN_ANIMATED_GRAPH = 40
    # cross correlation of interest rate against bankruptcies
    correlation = []

    def __init__(self, in_model):
        self.detailed_equity = None
        self.model = in_model

    def set_gif_graph(self, gif_graph):
        if gif_graph:
            self.create_gif = True

    def reset(self, output_directory=None):
        if output_directory:
            self.OUTPUT_DIRECTORY = output_directory
        if not os.path.isdir(self.OUTPUT_DIRECTORY):
            os.mkdir(self.OUTPUT_DIRECTORY)
        self.best_lender = np.full(self.model.config.T, -1, dtype=int)
        self.best_lender_clients = np.zeros(self.model.config.T, dtype=int)
        self.potential_credit_channels = np.zeros(self.model.config.T, dtype=int)
        self.active_borrowers = np.zeros(self.model.config.T, dtype=int)
        self.prob_bankruptcy = np.zeros(self.model.config.T, dtype=float)
        self.active_lenders = np.zeros(self.model.config.T, dtype=int)
        self.potential_lenders = np.zeros(self.model.config.T, dtype=int)
        self.fitness = np.zeros(self.model.config.T, dtype=float)
        self.c = np.zeros(self.model.config.T, dtype=float)
        self.interest_rate = np.zeros(self.model.config.T, dtype=float)
        self.asset_i = np.zeros(self.model.config.T, dtype=float)
        self.asset_j = np.zeros(self.model.config.T, dtype=float)
        self.equity = np.zeros(self.model.config.T, dtype=float)
        self.equity_borrowers = np.zeros(self.model.config.T, dtype=float)
        self.incrementD = np.zeros(self.model.config.T, dtype=float)
        self.liquidity = np.zeros(self.model.config.T, dtype=float)
        self.rationing = np.zeros(self.model.config.T, dtype=float)
        self.leverage = np.zeros(self.model.config.T, dtype=float)
        self.systemic_leverage = np.zeros(self.model.config.T, dtype=float)
        self.policy = np.zeros(self.model.config.T, dtype=float)
        self.P = np.zeros(self.model.config.T, dtype=float)
        self.P_max = np.zeros(self.model.config.T, dtype=float)
        self.P_min = np.zeros(self.model.config.T, dtype=float)
        self.P_std = np.zeros(self.model.config.T, dtype=float)
        self.B = np.zeros(self.model.config.T, dtype=float)
        self.loans = np.zeros(self.model.config.T, dtype=float)
        self.deposits = np.zeros(self.model.config.T, dtype=float)
        self.reserves = np.zeros(self.model.config.T, dtype=float)
        self.num_banks = np.zeros(self.model.config.T, dtype=int)
        self.bankruptcy = np.zeros(self.model.config.T, dtype=int)
        self.bankruptcy_rationed = np.zeros(self.model.config.T, dtype=int)
        self.num_of_rationed = np.zeros(self.model.config.T, dtype=int)
        self.psi = np.zeros(self.model.config.T, dtype=float)
        self.psi_lenders = np.zeros(self.model.config.T, dtype=float)

    def compute_credit_channels_and_best_lender(self):
        lenders = {}
        for bank in self.model.banks:
            if bank.lender is not None:
                if bank.lender in lenders:
                    lenders[bank.lender] += 1
                else:
                    lenders[bank.lender] = 1
        best = -1
        best_value = -1
        for lender in lenders.keys():
            if lenders[lender] > best_value:
                best = lender
                best_value = lenders[lender]
        self.best_lender[self.model.t] = best
        self.best_lender_clients[self.model.t] = best_value
        credit_channels = self.model.config.lender_change.get_credit_channels()
        if credit_channels is None:
            self.potential_credit_channels[self.model.t] = len(self.model.banks)
        else:
            self.potential_credit_channels[self.model.t] = credit_channels

    def compute_potential_lenders(self):
        for bank in self.model.banks:
            if bank.incrD>0:
                self.potential_lenders[self.model.t] += 1


    def compute_interest_rates_and_loans(self):
        interests_rates_of_borrowers = []
        psi_of_lenders = []
        asset_i = []
        asset_j = []
        sum_of_loans = 0
        num_of_banks_that_are_lenders = 0
        num_of_banks_that_are_borrowers = 0
        for bank in self.model.banks:            
            if bank.incrD >= 0:
                if bank.active_borrowers:
                    asset_i.append(bank.asset_i)
            elif bank.d > 0:
                asset_j.append(bank.asset_j)
            if bank.active_borrowers:
                num_of_banks_that_are_lenders += 1
                for bank_that_is_borrower in bank.active_borrowers:
                    sum_of_loans += bank.active_borrowers[bank_that_is_borrower]
                psi_of_lenders.append(bank.psi)
            elif bank.l > 0:
                num_of_banks_that_are_borrowers += 1
                interests_rates_of_borrowers.append(bank.get_loan_interest())
            bank.has_a_loan = bank.get_loan_interest() is not None and bank.l > 0

        self.interest_rate[self.model.t] = np.mean(interests_rates_of_borrowers) \
            if interests_rates_of_borrowers else 0.0
        self.asset_i[self.model.t] = np.mean(asset_i) if asset_i else np.nan
        self.asset_j[self.model.t] = np.mean(asset_j) if asset_j else np.nan
        if self.model.config.psi_endogenous:
            self.psi_lenders[self.model.t] = np.mean(psi_of_lenders) if psi_of_lenders else 0.0
            self.psi[self.model.t] = sum(bank.psi for bank in self.model.banks) / len(self.model.banks)
        else:
            self.psi_lenders[self.model.t] = self.model.config.psi
            self.psi[self.model.t] = self.model.config.psi
        self.loans[self.model.t] = sum_of_loans / num_of_banks_that_are_lenders \
            if num_of_banks_that_are_lenders else np.nan
        self.active_lenders[self.model.t] = num_of_banks_that_are_lenders
        self.active_borrowers[self.model.t] = num_of_banks_that_are_borrowers

    def compute_leverage_and_equity(self):
        sum_of_equity = 0
        sum_of_equity_borrowers = 0
        leverage_of_lenders = []
        self.model.statistics.save_detailed_equity(self.model.t)
        for bank in self.model.banks:
            if not bank.failed:
                sum_of_equity += bank.E
                if bank.l == 0:
                    amount_of_loan = 0
                    if bank.get_lender() is not None and bank.get_lender().l > 0:
                        amount_of_loan = bank.get_lender().l
                    leverage_of_lenders.append(amount_of_loan/ bank.E)
                if bank.active_borrowers:
                    for borrower in bank.active_borrowers:
                        sum_of_equity_borrowers += self.model.banks[borrower].E
                self.model.statistics.save_detailed_equity(bank.E)
            else:
                self.model.statistics.save_detailed_equity('')
        self.model.statistics.save_detailed_equity('\n')
        self.equity[self.model.t] = sum_of_equity
        self.equity_borrowers[self.model.t] = sum_of_equity_borrowers
        self.leverage[self.model.t] = np.mean(leverage_of_lenders) if leverage_of_lenders else 0.0
        # systemic_leverage = how the system is in relation to the total population of banks (big value  of 10 borrowers
        # against a population of 100 banks means that there is a risk
        self.systemic_leverage[self.model.t] = sum(leverage_of_lenders) / len(self.model.banks) \
            if len(self.model.banks) > 0 else 0

    def compute_liquidity(self):
        total_liquidity = 0
        for bank in self.model.banks:
            if not bank.failed:
                total_liquidity += bank.C
        self.liquidity[self.model.t] = total_liquidity

    def compute_fitness(self):
        self.fitness[self.model.t] = np.nan
        if self.model.config.N > 0:
            total_fitness = 0
            num_items = 0
            for bank in self.model.banks:
                if not bank.failed:
                    total_fitness += bank.mu
                    num_items += 1
            self.fitness[self.model.t] = total_fitness / num_items if num_items > 0 else np.nan

    def compute_policy(self):
        self.policy[self.model.t] = self.model.eta

    def compute_bad_debt(self):
        self.B[self.model.t] = sum((bank.B for bank in self.model.banks))

    def compute_rationing(self):
        self.rationing[self.model.t] = sum((bank.rationing for bank in self.model.banks))

    def compute_deposits_and_reserves(self):
        total_deposits = 0
        total_reserves = 0
        for bank in self.model.banks:
            if not bank.failed:
                total_deposits += bank.D
                total_reserves += bank.R
        self.deposits[self.model.t] = total_deposits
        self.reserves[self.model.t] = total_reserves

    def compute_probability_of_lender_change_num_banks_prob_bankruptcy(self):
        self.model.maxE = 0
        for bank in self.model.banks:
            if bank.E > self.model.maxE:
                self.model.maxE = bank.E
        avg_prob_bankruptcy = []
        if self.model.maxE > 0:
            for bank in self.model.banks:
                if bank.has_a_loan:
                    avg_prob_bankruptcy.append(1 - bank.E / self.model.maxE)
        self.prob_bankruptcy[self.model.t] = np.mean(avg_prob_bankruptcy) if avg_prob_bankruptcy else np.nan

        probabilities = []
        lender_capacities = []
        num_banks = 0
        for bank in self.model.banks:
            if not bank.failed:
                probabilities.append(bank.P)
                lender_capacities.append(np.mean(bank.c))
                num_banks += 1
        self.P[self.model.t] = sum(probabilities) / len(probabilities) if len(probabilities) > 0 else np.nan
        self.P_max[self.model.t] = max(probabilities) if probabilities else np.nan
        self.c[self.model.t] = np.mean(lender_capacities) if lender_capacities else np.nan
        self.P_min[self.model.t] = min(probabilities) if probabilities else np.nan
        self.P_std[self.model.t] = np.std(probabilities) if probabilities else np.nan
        self.num_banks[self.model.t] = num_banks

    def get_cross_correlation_result(self, t):
        if t in [0,1] and len(self.correlation)>t:
            status = '  '
            if self.correlation[t][0]>0:
                if self.correlation[t][1]<0.05:
                    status = '**'
                elif self.correlation[t][1]<0.10:
                    status = '* '
            return (f'correl t={t} int_rate/bankrupt {self.correlation[t][0]:4.2} '
                    f'p_value={self.correlation[t][1]:4.2} {status}')
        else:
            return " "

    def determine_cross_correlation(self):
        if not (self.bankruptcy == 0).all():
            self.correlation = [
                # correlation_coefficient = [-1..1] and p_value < 0.10
                scipy.stats.pearsonr(self.interest_rate,self.bankruptcy),
                # time delay 1:
                scipy.stats.pearsonr(self.interest_rate[1:],self.bankruptcy[:-1])
                ]
        else:
            self.correlation = []

    def export_data(self, export_datafile=None, export_description=None, generate_plots=True):
        if export_datafile:
            self.save_data(export_datafile, export_description)
            if generate_plots:
                self.get_plots(export_datafile)
        if Utils.is_notebook() or Utils.is_spyder():
            self.get_plots(None)

    def get_graph(self, t):
        """
        Extracts from the model the graph that corresponds to the network in this instant
        """
        if 'unittest' in sys.modules.keys():
            return None
        else:
            self.graphs[t] = nx.DiGraph(directed=True)
            for bank in self.model.banks:
                if bank.lender is not None:
                    self.graphs[t].add_edge(bank.lender, bank.id)
            draw(self.graphs[t], new_guru_look_for=True, title='t={}'.format(t))
            if Utils.is_spyder():
                plt.show()
                filename = None
            else:
                filename = sys.argv[0] if self.model.export_datafile is None else self.model.export_datafile
                filename = self.get_export_path(filename, '_{}{}'.format(t, self.graph_format))
                plt.savefig(filename)
            plt.close()
            return filename

    def define_plot_format(self, plot_format):
        match plot_format.lower():
            case 'none':
                self.plot_format = None
            case 'svg':
                self.plot_format = '.svg'
            case 'png':
                self.plot_format = '.png'
            case 'gif':
                self.plot_format = '.gif'
            case 'pdf':
                self.plot_format = '.pdf'
            case 'agr':
                self.plot_format = '.agr'
            case _:
                print('Invalid plot file format: {}'.format(plot_format))
                sys.exit(-1)

    def define_output_format(self, output_format):
        match output_format.lower():
            case 'both':
                self.output_format = '.both'
            case 'gdt':
                self.output_format = '.gdt'
            case 'csv':
                self.output_format = '.csv'
            case 'txt':
                self.output_format = '.txt'
            case _:
                print('Invalid output file format: {}'.format(output_format))
                sys.exit(-1)

    def create_gif_with_graphs(self, list_of_files):
        if len(list_of_files) == 0 or not self.create_gif:
            return
        else:
            if len(list_of_files) > self.NUMBER_OF_ITEMS_IN_ANIMATED_GRAPH:
                positions_of_images = len(list_of_files) / self.NUMBER_OF_ITEMS_IN_ANIMATED_GRAPH
            else:
                positions_of_images = 1
            filename_output = sys.argv[0] if self.model.export_datafile is None else self.model.export_datafile
            filename_output = self.get_export_path(filename_output, '.gif')
            images = []
            from PIL import Image
            for idx, image_file in enumerate(list_of_files):
                if not idx % positions_of_images == 0:
                    continue
                images.append(Image.open(image_file))
            images[0].save(fp=filename_output, format='GIF', append_images=images[1:], save_all=True, duration=100, loop=0)

    def get_export_path(self, filename, ending_name=''):
        if not os.path.dirname(filename):
            filename = '{}/{}'.format(self.OUTPUT_DIRECTORY, filename)
        path, extension = os.path.splitext(filename)
        if ending_name:
            return path + ending_name
        else:
            return path + self.output_format.lower()

    def __generate_csv_or_txt(self, export_datafile, header, delimiter):
        with open(export_datafile, 'w', encoding='utf-8') as savefile:
            for line_header in header:
                savefile.write('# {}\n'.format(line_header))
            savefile.write("# pd.read_csv('file{}',header={}', delimiter='{}')\nt".format(self.output_format, len(header) + 1, delimiter))
            for element_name, _ in self.enumerate_statistics_results():
                savefile.write('{}{}'.format(delimiter, element_name))
            savefile.write('\n')
            for i in range(self.model.config.T):
                savefile.write('{}'.format(i))
                for _, element in self.enumerate_statistics_results():
                    savefile.write('{}{}'.format(delimiter, element[i]))
                savefile.write('\n'.format())

    def __generate_gdt_file(self, filename, enumerate_results, header):
        E = lxml.builder.ElementMaker()
        gretl_data = E.gretldata
        DESCRIPTION = E.description
        VARIABLES = E.variables
        VARIABLE = E.variable
        OBSERVATIONS = E.observations
        OBS = E.obs
        variables = VARIABLES(count='{}'.format(sum((1 for _ in enumerate_results()))))
        header_text = ''
        for item in header:
            header_text += item + ' '
        # header_text will be present as label in the first variable
        # correlation_result will be present as label in the second variable
        i = 1
        for variable_name, _ in enumerate_results():
            if variable_name == 'leverage':
                variable_name += '_'
            if i==1:
                variables.append(VARIABLE(name='{}'.format(variable_name), label='{}'.format(header_text)))
            elif i in [2,3]:
                variables.append(VARIABLE(name='{}'.format(variable_name),
                        label=self.get_cross_correlation_result(i-2)))
            else:
                variables.append(VARIABLE(name='{}'.format(variable_name)))
            i = i+1
        observations = OBSERVATIONS(count='{}'.format(self.model.config.T), labels='false')
        for i in range(self.model.config.T):
            string_obs = ''
            for _, variable in enumerate_results():
                string_obs += '{}  '.format(variable[i])
            observations.append(OBS(string_obs))
        gdt_result = gretl_data(DESCRIPTION(header_text), variables, observations, version='1.4', name='interbank', frequency='special:1', startobs='1', endobs='{}'.format(self.model.config.T), type='time-series')
        with gzip.open(filename, 'w') as output_file:
            output_file.write(b'<?xml version="1.0" encoding="UTF-8"?>\n<!DOCTYPE gretldata SYSTEM "gretldata.dtd">\n')
            output_file.write(lxml.etree.tostring(gdt_result, pretty_print=True, encoding=str).encode('ascii'))

    def __generate_gdt(self, export_datafile, header):
        self.__generate_gdt_file(export_datafile, self.enumerate_statistics_results, header)

    @staticmethod
    def __transform_line_from_string(line_with_values):
        items = []
        for i in line_with_values.replace('  ', ' ').strip().split(' '):
            try:
                items.append(int(i))
            except ValueError:
                items.append(float(i))
        return items

    @staticmethod
    def read_gdt(filename):
        tree = lxml.etree.parse(filename)
        root = tree.getroot()
        children = root.getchildren()
        values = []
        columns = []
        if len(children) == 3:
            for variable in children[1].getchildren():
                column_name = variable.values()[0].strip()
                if column_name == 'leverage_':
                    column_name = 'leverage'
                columns.append(column_name)
            for value in children[2].getchildren():
                values.append(Statistics.__transform_line_from_string(value.text))
        if columns and values:
            return pd.DataFrame(columns=columns, data=values)
        else:
            return pd.DataFrame()

    def save_data(self, export_datafile=None, export_description=None):
        if export_datafile:
            if export_description:
                header = ['{}'.format(export_description)]
            else:
                header = ['{} T={} N={}'.format(__name__, self.model.config.T, self.model.config.N)]
            if self.output_format.lower() == '.both':
                self.output_format = '.csv'
                self.__generate_csv_or_txt(self.get_export_path(export_datafile), header, ';')
                self.output_format = '.gdt'
                self.__generate_gdt(self.get_export_path(export_datafile), header)
            elif self.output_format.lower() == '.csv':
                self.__generate_csv_or_txt(self.get_export_path(export_datafile), header, ';')
            elif self.output_format.lower() == '.txt':
                self.__generate_csv_or_txt(self.get_export_path(export_datafile), header, '\t')
            else:
                self.__generate_gdt(self.get_export_path(export_datafile), header)

    def enumerate_statistics_results(self):
        for element in Config.ELEMENTS_STATISTICS:
            yield (self.get_name(element), getattr(self, element))

    def get_name(self, variable):
        match variable:
            case 'bankruptcy':
                return 'bankruptcies'
            case 'P':
                return 'prob_change_lender'
            case 'B':
                return 'bad_debt'
        return variable

    def get_data(self):
        result = pd.DataFrame()
        for variable_name, variable in self.enumerate_statistics_results():
            result[variable_name] = np.array(variable)
        return result.iloc[0:self.model.t]

    def plot_pygrace(self, xx, yy_s, variable, title, export_datafile, x_label, y_label):
        import pygrace.project
        plot = pygrace.project.Project()
        graph = plot.add_graph()
        graph.title.text = title.capitalize().replace('_', ' ')
        for yy, color, ticks, title_y in yy_s:
            data = []
            if isinstance(yy, tuple):
                for i in range(len(yy[0])):
                    data.append((yy[0][i], yy[1][i]))
            else:
                for i in range(len(xx)):
                    data.append((xx[i], yy[i]))
            dataset = graph.add_dataset(data, legend=title_y)
            dataset.symbol.fill_color = color
        graph.xaxis.label.text = x_label
        graph.yaxis.label.text = y_label
        graph.set_world_to_limits()
        graph.autoscale()
        if export_datafile:
            if self.plot_format:
                plot.saveall(self.get_export_path(export_datafile, '_{}{}'.format(variable.lower(), self.plot_format)))

    def plot_pyplot(self, xx, yy_s, variable, title, export_datafile, x_label, y_label):
        if self.plot_format == '.agr':
            self.plot_pygrace(xx, yy_s, variable, title, export_datafile, x_label, y_label)
        else:
            plt.clf()
            plt.figure(figsize=(14, 6))
            for yy, color, ticks, title_y in yy_s:
                if isinstance(yy, tuple):
                    plt.plot(yy[0], yy[1], ticks, color=color, label=title_y, linewidth=0.2)
                else:
                    plt.plot(xx, yy, ticks, color=color, label=title_y)
            plt.xlabel(x_label)
            if y_label:
                plt.ylabel(y_label)
            plt.title(title.capitalize().replace('_', ' '))
            if len(yy_s) > 1:
                plt.legend()
            if export_datafile:
                if self.plot_format:
                    plt.savefig(self.get_export_path(export_datafile, '_{}{}'.format(variable.lower(), self.plot_format)))
            else:
                plt.show()
            plt.close()

    def plot_result(self, variable, title, export_datafile=None):
        xx = []
        yy = []
        for i in range(self.model.config.T):
            xx.append(i)
            yy.append(getattr(self, variable)[i])
        self.plot_pyplot(xx, [(yy, 'blue', '-', '')], variable, title, export_datafile, 'Time', '')

    def get_plots(self, export_datafile):
        for variable in Config.ELEMENTS_STATISTICS:
            if Config.ELEMENTS_STATISTICS[variable]:
                if 'plot_{}'.format(variable) in dir(Statistics):
                    eval('self.plot_{}(export_datafile)'.format(variable))
                else:
                    self.plot_result(variable, self.get_name(variable), export_datafile)

    def plot_P(self, export_datafile=None):
        xx = []
        yy = []
        yy_min = []
        yy_max = []
        yy_std = []
        for i in range(self.model.config.T):
            xx.append(i)
            yy.append(self.P[i])
            yy_min.append(self.P_min[i])
            yy_max.append(self.P_max[i])
            yy_std.append(self.P_std[i])
        self.plot_pyplot(xx, [(yy, 'blue', '-', 'Avg prob with $\\gamma$'), (yy_min, 'cyan', ':', 'Max and min prob'), (yy_max, 'cyan', ':', ''), (yy_std, 'red', '-', 'Std')], 'prob_change_lender', 'Prob of change lender ' + self.model.config.lender_change.describe(), export_datafile, 'Time', '')

    def plot_num_banks(self, export_datafile=None):
        if not self.model.config.allow_replacement_of_bankrupted:
            self.plot_result('num_banks', self.get_name('num_banks'), export_datafile)

    def plot_best_lender(self, export_datafile=None):
        xx = []
        yy = []
        yy2 = []
        max_duration = 0
        final_best_lender = -1
        current_lender = -1
        current_duration = 0
        time_init = 0
        for i in range(self.model.config.T):
            xx.append(i)
            yy.append(self.best_lender[i] / self.model.config.N)
            yy2.append(self.best_lender_clients[i] / self.model.config.N)
            if current_lender != self.best_lender[i]:
                if max_duration < current_duration:
                    max_duration = current_duration
                    time_init = i - current_duration
                    final_best_lender = current_lender
                current_lender = self.best_lender[i]
                current_duration = 0
            else:
                current_duration += 1
        xx3 = []
        yy3 = []
        for i in range(time_init, time_init + max_duration):
            xx3.append(i)
            yy3.append(self.best_lender[i] / self.model.config.N)
        self.plot_pyplot(xx, [(yy, 'blue', '-', 'id'), (yy2, 'red', '-', 'Num clients'), ((xx3, yy3), 'orange', '-', '')], 'best_lender', 'Best Lender (blue) #clients (red)', export_datafile, 'Time (best lender={} at t=[{}..{}])'.format(final_best_lender, time_init, time_init + max_duration), 'Best Lender')

    def enable_detailed_equity(self):
        self.define_detailed_equity(self.model.export_datafile)

    def define_detailed_equity(self, detailed_equity, save_file=None):
        if detailed_equity:
            save_file = save_file + '_equity.csv' if save_file else 'equity.csv'
            self.detailed_equity = open(self.get_export_path(save_file).replace('.gdt', '.csv'), 'w')
            self.save_detailed_equity('t')
            for i in range(self.model.config.N):
                self.save_detailed_equity('bank_{}'.format(i))
            self.save_detailed_equity('\n')

    def save_detailed_equity(self, value):
        if self.detailed_equity:
            if isinstance(value, int):
                value = '{};'.format(value)
            elif isinstance(value, float):
                value = '{};'.format(value)
            elif value != '\n':
                value = '{};'.format(value)
            self.detailed_equity.write(value)

In [ ]:
class Log:
    """
    The class acts as a logger and helpers to represent the data and evol from the Model.
    """
    logger = logging.getLogger('model')
    modules = []
    model = None
    logLevel = 'ERROR'
    progress_bar = None

    def __init__(self, its_model):
        self.model = its_model

    def do_progress_bar(self, message, maximum):
        from progress.bar import Bar
        self.progress_bar = Bar(message, max=maximum)

    @staticmethod
    def __format_number__(number):
        result = '{}'.format(number)
        while len(result) > 5 and result[-1] == '0':
            result = result[:-1]
        while len(result) > 5 and result.find('.') > 0:
            result = result[:-1]
        return result

    def debug_banks(self, details: bool=True, info: str=''):
        for bank in self.model.banks:
            if not info:
                info = '-----'
            self.info(info, bank.__str__(details=details))

    @staticmethod
    def get_level(option):
        try:
            return getattr(logging, option.upper())
        except AttributeError:
            logging.error(" '--log' must contain a valid logging level and {} is not.".format(option.upper()))
            sys.exit(-1)

    def debug(self, module, text):
        if self.modules == [] or module in self.modules:
            if text:
                self.logger.debug('t={}/{} {}'.format(self.model.t, module, text))

    def info(self, module, text):
        if self.modules == [] or module in self.modules:
            if text:
                self.logger.info(' t={}/{} {}'.format(self.model.t, module, text))

    def error(self, module, text):
        if text:
            self.logger.error('t={}/{} {}'.format(self.model.t, module, text))

    def define_log(self, log: str, logfile: str='', modules: str='', script_name: str='%(module)s'):
        self.modules = modules.split(',') if modules else []
        formatter = logging.Formatter('%(levelname)s-' + '- %(message)s')
        self.logLevel = Log.get_level(log.upper())
        self.logger.setLevel(self.logLevel)
        if logfile:
            if not os.path.dirname(logfile):
                logfile = '{}/{}'.format(self.model.statistics.OUTPUT_DIRECTORY, logfile)
            fh = logging.FileHandler(logfile, 'a', 'utf-8')
            fh.setLevel(self.logLevel)
            fh.setFormatter(formatter)
            self.logger.addHandler(fh)
        else:
            ch = logging.StreamHandler()
            ch.setLevel(self.logLevel)
            ch.setFormatter(formatter)
            self.logger.addHandler(ch)

In [ ]:
class Model:
    """
    It contains the banks and has the logic to execute the simulation
        import interbank
        model = interbank.Model( )
        model.configure( param=x )
        model.forward()
        μ = model.get_current_fitness()
        model.set_policy_recommendation( ŋ=0.5 )
    or
        model.set_policy_recommendation( 1 ) --> equal to n=0.5, because values are int 0,1,2 ---> float 0,0.5,1

    If you want to forward() and backward() step by step, you should use:
        model.configure( backward=True )
        # t=4
        model.set_policy_recommendation( ŋ=0.5 )
        model.forward()
        result = model.get_current_fitness() # t=5
        model.backward() # t=4 again
    """
    banks = []
    t: int = 0
    eta: float = 1
    test = False
    default_seed: int = 20579
    backward_enabled = False
    policy_changes = 0
    save_graphs = None
    save_graphs_results = []
    log = None
    maxE = Config.E_i0
    statistics = None
    config = None
    export_datafile = None
    export_description = None
    policy_actions_translation = [0.0, 0.5, 1.0]
    generate_plots = True

    def __init__(self, **configuration):
        self.log = Log(self)
        self.statistics = Statistics(self)
        self.config = Config()
        self.value_for_reintroduced_banks_L = self.config.L_i0
        self.value_for_reintroduced_banks_E = self.config.E_i0
        self.value_for_reintroduced_banks_D = self.config.D_i0
        if configuration:
            self.configure(**configuration)
        if self.backward_enabled:
            self.banks_backward_copy = []

    def configure_json(self, json_string: str):
        import re, json
        json_string = json_string.strip().replace('=', ':').replace(' ', ', ').replace('True', 'true').replace('False', 'false')
        if not json_string.startswith('{'):
            json_string = '{' + json_string
        if not json_string.endswith('}'):
            json_string += '}'
        self.configure(**json.loads(re.sub('(?<=\\{|\\s)(\\w+)(?=\\s*:)', '"\\1"', json_string)))

    def configure(self, **configuration):
        for attribute in configuration:
            if attribute.startswith('lc'):
                attribute = attribute.replace('lc_', '')
                if attribute == 'lc':
                    self.config.lender_change = determine_algorithm(configuration[attribute])
                else:
                    self.config.lender_change.set_parameter(attribute, configuration['lc_' + attribute])
            elif hasattr(self.config, attribute):
                setattr(self.config, attribute, configuration[attribute])
            else:
                raise LookupError('attribute in config not found: %s ' % attribute)

    def initialize(self, seed=None, dont_seed=False, save_graphs_instants=None, export_datafile=None,
                   export_description=None, generate_plots=True, output_directory=None):
        self.statistics.reset(output_directory=output_directory)
        if not seed is None and not dont_seed:
            applied_seed = seed if seed else self.default_seed
            random.seed(applied_seed)
            self.config.seed = applied_seed
        self.save_graphs = save_graphs_instants
        self.banks = []
        self.t = 0
        if not self.config.lender_change:
            self.config.lender_change = determine_algorithm()
            self.config.lender_change.set_parameter('p', 0.5)
        self.policy_changes = 0
        if export_datafile:
            self.export_datafile = export_datafile
        if generate_plots:
            self.generate_plots = generate_plots
        if export_description is None:
            self.export_description = str(self.config) + str(self.config.lender_change)
        else:
            self.export_description = export_description
        for i in range(self.config.N):
            self.banks.append(Bank(i, self))
        self.config.lender_change.initialize_bank_relationships(self)

    def forward(self):
        self.initialize_step()
        if self.backward_enabled:
            self.banks_backward_copy = copy.deepcopy(self.banks)
        self.do_shock('shock1')
        self.statistics.compute_potential_lenders()
        self.do_loans()
        self.log.debug_banks()
        self.statistics.compute_interest_rates_and_loans()
        self.statistics.compute_leverage_and_equity()
        self.do_shock('shock2')
        self.do_repayments()
        self.log.debug_banks()
        if self.log.progress_bar:
            self.log.progress_bar.next()
        self.statistics.compute_liquidity()
        self.statistics.compute_credit_channels_and_best_lender()
        self.statistics.compute_fitness()
        self.statistics.compute_policy()
        self.statistics.compute_deposits_and_reserves()
        self.statistics.bankruptcy_rationed[self.t] = self.replace_bankrupted_banks()
        self.setup_links()
        self.statistics.compute_probability_of_lender_change_num_banks_prob_bankruptcy()
        self.log.debug_banks()
        if self.save_graphs is not None and (self.save_graphs == '*' or self.t in self.save_graphs):
            filename = self.statistics.get_graph(self.t)
            if filename:
                self.save_graphs_results.append(filename)
        self.t += 1

    def backward(self):
        if self.backward_enabled:
            if self.t > 0:
                self.banks = self.banks_backward_copy
                self.t -= 1
            else:
                raise IndexError('t=0 and no backward is possible')
        else:
            raise AttributeError('enable_backward() before')

    def enable_backward(self):
        self.backward_enabled = True

    def simulate_full(self, interactive=False):
        if interactive:
            self.log.do_progress_bar('Simulating t=0..{}'.format(self.config.T), self.config.T)
        for t in range(self.config.T):
            self.forward()
            if not self.config.allow_replacement_of_bankrupted and len(self.banks) <= 2:
                self.config.T = self.t
                self.log.debug('*****', 'Finish because there are only two banks surviving'.format())
                break


    def finish(self):
        if not self.test:
            self.statistics.determine_cross_correlation()
            self.statistics.export_data(export_datafile=self.export_datafile,
                                        export_description=self.export_description,
                                        generate_plots=self.generate_plots)
        summary = 'Finish: model T={}  N={}'.format(self.config.T, self.config.N)
        if not self.__policy_recommendation_changed__():
            summary += ' ŋ={}'.format(self.eta)
        else:
            summary += ' ŋ variate during simulation'
        self.log.info('*****', summary)
        self.statistics.create_gif_with_graphs(self.save_graphs_results)
        plt.close()
        return self.statistics.get_data()

    def set_policy_recommendation(self, n: int=None, eta: float=None, eta_1: float=None):
        if eta_1 is not None:
            n = round(eta_1)
        if n is not None and eta is None:
            if type(n) is int:
                eta = self.policy_actions_translation[n]
            else:
                eta = float(n)
        if self.eta != eta:
            self.log.debug('*****', 'eta(ŋ) changed to {}'.format(eta))
            self.policy_changes += 1
        self.eta = eta

    def limit_to_two_policies(self):
        self.policy_actions_translation = [0.0, 1.0]

    def __policy_recommendation_changed__(self):
        return self.policy_changes > 1

    def get_current_fitness(self):
        """
        Determines the current μ of the model (does the sum of all μ)
        :return:
        float:  Ʃ banks.μ
        """
        return self.statistics.fitness[self.t - 1 if self.t > 0 else 0]

    def get_current_liquidity(self):
        """
        Returns the liquidity (the sum of the liquidity)
        :return:
        float:  Ʃ banks.C
        """
        return self.statistics.liquidity[self.t - 1 if self.t > 0 else 0]

    def get_current_interest_rate(self):
        """
        Returns the interest rate (the average of all banks)
        :return:
        float:  Ʃ banks.ir / config.N
        """
        return self.statistics.interest_rate[self.t - 1 if self.t > 0 else 0]

    def get_current_interest_rate_info(self):
        """
        Returns a tuple with  : max ir, min_ir and avg
        :return:
        (float,float,float)
        """
        max_ir = 0
        min_ir = np.inf
        for bank in self.banks:
            bank_ir = bank.get_loan_interest()
            if bank_ir is not None:
                if max_ir < bank_ir:
                    max_ir = bank_ir
                if min_ir > bank_ir:
                    min_ir = bank_ir
        return (max_ir, min_ir, self.get_current_interest_rate())

    def get_current_liquidity_info(self):
        """
        Returns a tuple with  : max C, min C and avg C
        :return:
        (float,float,float)
        """
        max_c = 0
        min_c = 1000000.0
        for bank in self.banks:
            bank_c = bank.C
            if max_c < bank_c:
                max_c = bank_c
            if min_c > bank_c:
                min_c = bank_c
        return (max_c, min_c, self.get_current_liquidity())

    def get_current_bankruptcies(self):
        """
        Returns the number of bankruptcies in this step
        :return:
        int:  Ʃ failed banks
        """
        return self.statistics.bankruptcy[self.t - 1 if self.t > 0 else 0]

    def determine_shock_value(self, bank, _shock):
        rand_value = random.random()
        return bank.D * (self.config.mu + self.config.omega * rand_value)

    def do_shock(self, which_shock):
        for bank in self.banks:
            bank.newD = self.determine_shock_value(bank, which_shock)
            bank.incrD = bank.newD - bank.D
            bank.D = bank.newD
            bank.newR = self.config.reserves * bank.D
            bank.incrR = bank.newR - bank.R
            bank.R = bank.newR
            if bank.incrD >= 0:
                bank.C += bank.incrD - bank.incrR
                if which_shock == 'shock1':
                    bank.s = bank.C
                bank.d = 0
                if bank.incrD > 0:
                    self.log.debug(which_shock, '{} wins ΔD={}'.format(bank.get_id(), bank.incrD))
                else:
                    self.log.debug(which_shock, '{} has no shock'.format(bank.get_id()))
            else:
                if which_shock == 'shock1':
                    bank.s = 0
                if bank.incrD - bank.incrR + bank.C >= 0:
                    bank.d = 0
                    bank.C += bank.incrD - bank.incrR
                    self.log.debug(which_shock, '{} loses ΔD={}, covered by capital'.format(bank.get_id(), bank.incrD))
                else:
                    bank.d = abs(bank.incrD - bank.incrR + bank.C)
                    self.log.debug(which_shock, '{} loses ΔD={} has C={} and needs {}'.format(bank.get_id(), bank.incrD, bank.C, bank.d))
                    if which_shock == 'shock2':
                        bank.do_fire_sales(bank.d, 'ΔD={},C=0 and we need {}'.format(bank.incrD, bank.d), which_shock)
                    else:
                        bank.C = 0
            self.statistics.incrementD[self.t] += bank.incrD

    def do_loans(self):
        self.config.lender_change.extra_relationships_change(self)

        # first we normalize the banks interest rate if it is necessary:
        if self.config.normalize_interest_rate_max and self.config.normalize_interest_rate_max>0:
            max_r = 0
            for bank in self.banks:
                if not bank.get_loan_interest() is None and bank.get_loan_interest() > max_r:
                    max_r = bank.get_loan_interest()
            if max_r > self.config.r_i0:
                for bank in self.banks:
                    if not bank.lender is None:
                        self.banks[bank.lender].rij[bank.id] = (
                                self.banks[bank.lender].rij[bank.id] / max_r * self.config.normalize_interest_rate_max)

        num_of_rationed = 0
        total_rationed = 0
        total_demanded = 0
        total_loans = 0
        for bank_index, bank in enumerate(self.banks):
            rationing_of_bank = 0
            lender = bank.get_lender()
            demand = bank.d
            if demand > 0:
                total_demanded += demand
                if lender is None or lender.d > 0:
                    bank.l = 0
                    rationing_of_bank = demand
                    total_rationed += rationing_of_bank
                    num_of_rationed += 1
                    bank.do_fire_sales(rationing_of_bank, 
                        f'rationing={{rationing_of_bank}} as no lender for this bank' if lender is None 
                        else f'rationing={{rationing_of_bank}} as lender {{lender.get_id(short=True)}} has no money', 
                        'loans')
                elif demand > lender.s:
                    rationing_of_bank = demand - lender.s
                    total_rationed += rationing_of_bank
                    num_of_rationed += 1
                    bank.do_fire_sales(rationing_of_bank,
                        f'lender.s={{lender.s}} but need d={{demand}}, rationing={{rationing_of_bank}}', 
                        'loans')
                    loan = lender.s if lender.s > 0 else 0
                    bank.l = loan
                    if loan > 0:
                        lender.active_borrowers[bank_index] = loan
                        lender.C -= loan
                        lender.s = 0
                    total_loans += loan
                else:
                    bank.l = demand
                    lender.active_borrowers[bank_index] = demand
                    lender.C -= demand
                    lender.s -= demand
                    total_loans += demand
            else:
                bank.l = 0
                if bank.active_borrowers:
                    pass  # can skip string building/logging
            bank.rationing = rationing_of_bank
        self.statistics.num_of_rationed[self.t] = num_of_rationed
        self.statistics.rationing[self.t] = total_rationed


    def do_repayments(self):
        for bank in self.banks:
            if bank.l > 0 and bank.d > 0 and (not bank.failed):
                amount_we_need = bank.l + bank.d - bank.C
                if amount_we_need > 0:
                    obtained = bank.do_fire_sales(amount_we_need, 'fire sales due to not enough C'.format(), 'repay')
                    bank.d -= obtained
                    if bank.d < 0:
                        bank.l += bank.d
                        bank.d = 0
                        if bank.l < 0 and (not bank.failed):
                            bank.do_bankruptcy('repay')
                bank.reviewed = True
            else:
                bank.reviewed = False
        for bank in self.banks:
            if bank.l > 0 and (not bank.reviewed) and (not bank.failed):
                loan_profits = bank.get_loan_interest() * bank.l
                loan_to_return = bank.l + loan_profits
                bank_lender = bank.get_lender()
                if loan_to_return > bank.C:
                    lack_of_capital_to_return_loan = loan_to_return - bank.C
                    bank.C = 0
                    obtained_in_fire_sales = bank.do_fire_sales(lack_of_capital_to_return_loan, 'to return loan and interest {} > C={}'.format(loan_to_return, bank.C), 'repay')
                    gap_of_money_not_covered_of_loan = lack_of_capital_to_return_loan - obtained_in_fire_sales
                    if gap_of_money_not_covered_of_loan > loan_profits:
                        bank.paid_profits = 0
                        bank.paid_loan = bank.l - gap_of_money_not_covered_of_loan + loan_profits
                    else:
                        bank.paid_loan = bank.l
                        bank.paid_profits = loan_profits - gap_of_money_not_covered_of_loan
                    if not bank_lender.failed and (not bank.failed):
                        bank_lender.C += bank.paid_loan
                        bank_lender.E += bank.paid_profits
                        del bank_lender.active_borrowers[bank.id]
                else:
                    bank.C -= loan_to_return
                    bank.paid_loan = bank.l
                    bank.paid_profits = loan_profits
                    bank_lender.C += bank.paid_loan
                    bank_lender.E += bank.paid_profits
                    del bank_lender.active_borrowers[bank.id]
                bank_lender.s += bank.paid_loan
                bank.E -= loan_profits
                if bank.E < 0:
                    bank.failed = True
                    self.log.debug('repay', '{} fails because the profits of the loan generates E<0'.format(bank.get_id()))
        for bank in self.banks:
            if bank.l == 0 and (not bank.failed):
                if bank.C < bank.incrD:
                    bank.d = bank.incrD - bank.C
                else:
                    bank.d = 0
                if bank.d > 0:
                    bank.do_fire_sales(bank.d, 'fire sales due to not enough C'.format(), 'repay')

    def replace_bankrupted_banks(self):
        self.estimate_average_values_for_replacement_of_banks()
        self.statistics.compute_bad_debt()
        lists_to_remove_because_replacement_of_bankrupted_is_disabled = []
        num_banks_failed_rationed = 0
        total_removed = 0
        for possible_removed_bank in self.banks:
            if possible_removed_bank.failed:
                total_removed += 1
                if self.config.allow_replacement_of_bankrupted:
                    possible_removed_bank.replace_bank()
                else:
                    if possible_removed_bank.rationing == 0 and possible_removed_bank.lender is not None:
                        num_banks_failed_rationed += 1
                    lists_to_remove_because_replacement_of_bankrupted_is_disabled.append(possible_removed_bank)
        self.log.debug('repay', 'this step ΔD={} and '.format(self.statistics.incrementD[self.t]) + 'failures={}'.format(total_removed))
        if not self.config.allow_replacement_of_bankrupted:
            for bank_to_remove in lists_to_remove_because_replacement_of_bankrupted_is_disabled:
                self.__remove_without_replace_failed_bank(bank_to_remove)
            self.config.N -= len(lists_to_remove_because_replacement_of_bankrupted_is_disabled)
            self.log.debug('repay', 'now we have {} banks'.format(self.config.N))
        return num_banks_failed_rationed

    def __remove_without_replace_failed_bank(self, bank_to_remove):
        self.banks.remove(bank_to_remove)
        self.log.debug('repay', '{} bankrupted and removed'.format(bank_to_remove.get_id()))
        for bank_i in self.banks:
            if bank_i.lender is None or bank_i.lender == bank_to_remove.id:
                bank_i.lender = None
            elif bank_i.lender > bank_to_remove.id:
                bank_i.lender -= 1
            if bank_i.id > bank_to_remove.id:
                bank_i.id -= 1
            for borrower in list(bank_i.active_borrowers):
                if borrower > bank_to_remove.id:
                    bank_i.active_borrowers[borrower - 1] = bank_i.active_borrowers[borrower]
                    del bank_i.active_borrowers[borrower]
                elif borrower == bank_to_remove.id:
                    del bank_i.active_borrowers[borrower]

    def initialize_step(self):
        for bank in self.banks:
            bank.B = 0
            bank.rationing = 0
            bank.paid_profits = 0
            bank.paid_loan = 0
            bank.active_borrowers = {}
        if self.t == 0:
            self.log.debug_banks()


    def setup_links(self):
        if len(self.banks) <= 1:
            return
        self.maxE = max(self.banks, key=lambda k: k.E).E
        maxC = max(self.banks, key=lambda k: k.C).C
        for bank in self.banks:
            bank.p = bank.E / self.maxE
            if bank.get_lender() is not None and bank.get_lender().l > 0:
                bank.lambda_ = bank.get_lender().l / bank.E
            else:
                bank.lambda_ = 0
            # bank.lambda_ = bank.l / bank.E
            bank.incrD = 0
        max_lambda = max(self.banks, key=lambda k: k.lambda_).lambda_
        for bank in self.banks:
            bank.h = bank.lambda_ / max_lambda if max_lambda > 0 else 0
            bank.A = bank.C + bank.L + bank.R
        for bank in self.banks:
            bank.c = []
            for i in range(self.config.N):
                c = 0 if i == bank.id else (1 - self.banks[i].h) * self.banks[i].A
                bank.c.append(c)
            if self.config.psi_endogenous:
                bank.psi = bank.E / self.maxE
        min_r = sys.maxsize
        for bank_i in self.banks:
            bank_i.asset_i = 0
            bank_i.asset_j = 0
            for j in range(self.config.N):
                try:
                    if j == bank_i.id:
                        bank_i.rij[j] = 0
                    else:
                        if self.banks[j].p == 0 or bank_i.c[j] == 0:
                            bank_i.rij[j] = self.config.r_i0
                        else:
                            psi = bank_i.psi if self.config.psi_endogenous else self.config.psi
                            if psi==1:
                                psi=0.99999999999999
                            bank_i.rij[j] = ((self.config.chi * bank_i.A - self.config.phi * self.banks[j].A
                                              - (1 - self.banks[j].p) * (self.config.xi * self.banks[j].A - bank_i.c[j]))
                                             /
                                             (self.banks[j].p * bank_i.c[j] * (1 - psi)))


                            bank_i.asset_i += bank_i.A
                            bank_i.asset_j += self.banks[j].A
                            # bank_i.asset_j += 1 - self.banks[j].p
                        if bank_i.rij[j] < 0:
                            bank_i.rij[j] = self.config.r_i0
                except ZeroDivisionError:
                    bank_i.rij[j] = self.config.r_i0
            bank_i.r = np.sum(bank_i.rij) / (self.config.N - 1)
            bank_i.asset_i = bank_i.asset_i / (self.config.N - 1)
            bank_i.asset_j = bank_i.asset_j / (self.config.N - 1)
            if bank_i.r < min_r:
                min_r = bank_i.r
        for bank in self.banks:
            bank.mu = self.eta * (bank.C / maxC) + (1 - self.eta) * (min_r / bank.r)
        self.config.lender_change.step_setup_links(self)
        for bank in self.banks:
            log_change_lender = self.config.lender_change.change_lender(self, bank, self.t)
            self.log.debug('links', log_change_lender)

    def estimate_average_values_for_replacement_of_banks(self):
        self.value_for_reintroduced_banks_L = self.config.L_i0
        self.value_for_reintroduced_banks_E = self.config.E_i0
        self.value_for_reintroduced_banks_D = self.config.D_i0
        if self.config.reintroduce_with_median:
            banks_L = []
            banks_E = []
            banks_D = []
            for bank in self.banks:
                if not bank.failed and bank.E > 0:
                    banks_L.append(bank.L)
                    banks_D.append(bank.D)
                    banks_E.append(bank.E)
            if banks_L:
                self.value_for_reintroduced_banks_L = np.median(banks_L)
                self.value_for_reintroduced_banks_D = np.median(banks_D)
                self.value_for_reintroduced_banks_E = np.median(banks_E)

In [ ]:
class Bank:
    """
    It represents an individual bank of the network, with the logic of interaction between it and the interbank system
    """

    def get_lender(self):
        if self.lender is None or self.lender >= len(self.model.banks):
            return None
        else:
            return self.model.banks[self.lender]

    def get_loan_interest(self):
        if self.lender is None or self.lender >= len(self.model.banks):
            return None
        else:
            return self.model.banks[self.lender].rij[self.id]
            #import math
            #return math.sqrt(math.sqrt(self.model.banks[self.lender].rij[self.id]))/20


    def get_id(self, short: bool=False):
        init = 'bank#' if not short else '#'
        if self.failures > 0:
            return '{}{}.{}'.format(init, self.id, self.failures)
        else:
            return '{}{}'.format(init, self.id)

    def __init__(self, new_id=None, bank_model=None):
        if not new_id is None and bank_model:
            self.id = new_id
            self.model = bank_model
            self.lender = None
            self.failures = 0
        self.L = self.model.value_for_reintroduced_banks_L
        self.D = self.model.value_for_reintroduced_banks_D
        self.E = self.model.value_for_reintroduced_banks_E
        self.A = 0
        self.r = 0
        self.rij: list[Any] = [0] * self.model.config.N
        self.c: list[Any] = []
        self.h = 0
        self.p = 0
        self.R = self.model.config.reserves * self.D
        self.C = self.D + self.E - self.L - self.R
        self.mu = 0
        self.l = 0
        self.s = 0
        self.d = 0
        self.B = 0
        self.psi = self.E / self.model.maxE if self.model.config.psi_endogenous else None
        self.incrD = 0
        self.paid_profits = 0
        self.paid_loan = 0
        self.incrR = 0
        self.rationing = 0
        self.lambda_ = 0
        self.failed = False
        self.lender = self.model.config.lender_change.new_lender(self.model, self)
        self.active_borrowers = {}
        self.asset_i = 0
        self.asset_j = 0

    def replace_bank(self):
        self.failures += 1
        self.__init__()

    def do_bankruptcy(self, phase):
        self.failed = True
        self.model.statistics.bankruptcy[self.model.t] += 1
        recovered_in_fire_sales = self.L * self.model.config.rho
        recovered = recovered_in_fire_sales - self.D
        if recovered < 0:
            recovered = 0
        if recovered > self.l:
            recovered = self.l
        bad_debt = self.l - recovered
        self.D = 0
        if not self.get_lender() is None and self.l > 0:
            if bad_debt > 0:
                self.get_lender().B += bad_debt
                self.get_lender().E -= bad_debt
                if self.get_lender().E < 0:
                    self.model.log.debug(phase, '{} lender is bankrupted  borrower {} does not return loan and lender E<0: {}'.format(self.get_lender().get_id(), self.get_id(), self.get_lender().E))
                    self.get_lender().failed = True
                self.get_lender().C += recovered
                self.model.log.debug(phase, '{} bankrupted (fire sale={},recovers={},paidD={})(lender{}.ΔB={},ΔC={})'.format(self.get_id(), recovered_in_fire_sales, recovered, self.D, self.get_lender().get_id(short=True), bad_debt, recovered))
            elif self.l > 0 and self.get_lender() is not None:
                self.get_lender().C += self.l
                self.model.log.debug(phase, '{} bankrupted (lender{}.ΔB=0,ΔC={}) (paidD={})'.format(self.get_id(), self.get_lender().get_id(short=True), recovered, self.l))
            self.get_lender().s += recovered
            if self.id in self.get_lender().active_borrowers:
                del self.get_lender().active_borrowers[self.id]
        return recovered

    def do_fire_sales(self, amount_to_sell, reason, phase):
        cost_of_sell = (amount_to_sell / self.model.config.rho) if self.model.config.rho else np.inf
        extra_cost_of_selling = cost_of_sell * (1 - self.model.config.rho)
        if cost_of_sell > self.L:
            self.model.log.debug(phase, '{} impossible fire sale to recover {}: cost_sell_L={} > L={}: {}'.format(self.get_id(), amount_to_sell, cost_of_sell, self.L, reason))
            return self.do_bankruptcy(phase)
        else:
            self.L -= cost_of_sell
            self.E -= extra_cost_of_selling
            self.model.log.debug(phase, '{} fire sales {} so L-={} and affects to E-={}'.format(self.get_id(), amount_to_sell, cost_of_sell, extra_cost_of_selling))
            if self.L <= self.model.config.alfa:
                self.model.log.debug(phase, '{} new L={} is under threshold {} and makes bankruptcy of bank: {}'.format(self.get_id(), self.L, self.model.config.alfa, reason))
                self.do_bankruptcy(phase)
                return amount_to_sell
            else:
                if self.E <= self.model.config.alfa:
                    self.model.log.debug(phase, '{} new E={} is under threshold {} and makes bankruptcy of bank: {}'.format(self.get_id(), self.E, self.model.config.alfa, reason))
                    self.do_bankruptcy(phase)
                return amount_to_sell

    def __str__(self, details=False):
        text = '{} C={} R={} L={}'.format(self.get_id(short=True), Log.__format_number__(self.C), Log.__format_number__(self.R), Log.__format_number__(self.L))
        amount_borrowed = 0
        list_borrowers = ' borrows=['
        for bank_i in self.active_borrowers:
            list_borrowers += self.model.banks[bank_i].get_id(short=True) + ','
            amount_borrowed += self.active_borrowers[bank_i]
        if amount_borrowed:
            text += ' l={}'.format(Log.__format_number__(amount_borrowed))
            list_borrowers = list_borrowers[:-1] + ']'
        else:
            text += '        '
            list_borrowers = ''
        text += ' | D={} E={}'.format(Log.__format_number__(self.D), Log.__format_number__(self.E))
        if details and hasattr(self, 'd') and self.d and self.l:
            text += ' l={}'.format(Log.__format_number__(self.d))
        else:
            text += '        '
        if details and hasattr(self, 's') and self.s:
            text += ' s={}'.format(Log.__format_number__(self.s))
        elif details and hasattr(self, 'd') and self.d:
            text += ' d={}'.format(Log.__format_number__(self.d))
        else:
            text += '        '
        if self.failed:
            text += ' FAILED '.format()
        elif details and hasattr(self, 'd') and (self.d > 0):
            if self.get_lender() is None:
                text += ' no lender'.format()
            else:
                text += ' lender{},r={}%'.format(self.get_lender().get_id(short=True), self.get_loan_interest())
        else:
            text += list_borrowers
        text += ' B={}'.format(Log.__format_number__(self.B)) if self.B else '        '
        if self.model.config.psi_endogenous and self.psi:
            text += ' psi={}'.format(Log.__format_number__(self.psi))
        return text

In [ ]:
class Utils:
    """
    Auxiliary class to encapsulate the use of the model
    """

    @staticmethod
    def __extract_t_values_from_arg__(param):
        if param is None:
            return None
        else:
            t = []
            if param == '*':
                return '*'
            else:
                for str_t in param.split(','):
                    t.append(int(str_t))
                    if t[-1] > Config.T or t[-1] < 0:
                        raise ValueError('{} greater than Config.T or below 0'.format(t[-1]))
                return t

    @staticmethod
    def run_interactive():
        """
            Run interactively the model
        """
        global model
        parser = argparse.ArgumentParser()
        parser.description = "<config=value> to set up Config options. use '?' to see values"
        parser.add_argument('--log', default='ERROR', help='Log level messages (ERROR,DEBUG,INFO...)')
        parser.add_argument('--modules', default=None,
                            help='Log only this modules (separated by ,)'.format())
        parser.add_argument('--logfile', default=None, help='File to send logs to')
        parser.add_argument('--save', default=None, help='Saves the output of this execution'.format())
        parser.add_argument('--graph', default=None,
                            help='List of t in which save the network config (* for all)'.format())
        parser.add_argument('--gif_graph', default=False, type=bool,
                            help='If --graph, then also an animated gif with all graphs '.format())
        parser.add_argument('--graph_stats', default=False, type=str,
                            help='Load a json of a graph and give us statistics of it'.format())
        parser.add_argument('--n', type=int, default=Config.N, help='Number of banks'.format())
        parser.add_argument('--eta', type=float, default=Model.eta, help='Policy recommendation'.format())
        parser.add_argument('--t', type=int, default=Config.T, help='Time repetitions'.format())
        parser.add_argument('--lc_p', '--p', type=float, default=LENDER_CHANGE_DEFAULT_P,
                            help="For Erdos-Renyi bank lender's change value of p".format())
        parser.add_argument('--lc_m', '--m', type=int, default=None,
                            help="For Preferential bank lender's change value of graph grade m".format())
        parser.add_argument('--lc', type=str, default=LENDER_CHANGE_DEFAULT,
                            help="Bank lender's change method (?=list)")
        parser.add_argument('--lc_ini_graph_file', type=str, default=None,
                            help='Load a graph in json networkx.node_link_data() format')
        parser.add_argument('--detailed_equity', action='store_true',
                            default=model.config.detailed_equity,
                            help='Store in a gdt the individual E evolution of each individual bank')
        parser.add_argument('--psi_endogenous', action='store_true',
                            default=model.config.psi_endogenous, help='Market power variable psi will be endogenous')
        parser.add_argument('--plot_format', type=str, default='none',
                            help='Generate plots with the specified format (svg,png,pdf,gif,agr)')
        parser.add_argument('--output_format', type=str, default='gdt',
                            help='File extension for data (gdt,txt,csv,both)')
        parser.add_argument('--output', type=str, default=None,

                            help='Directory where to store the results')
        parser.add_argument('--no_replace', action='store_true',
                            default=not model.config.allow_replacement_of_bankrupted,
                            help='No replace banks when they go bankrupted')
        parser.add_argument('--reintr_with_median', action='store_true',
                            default=model.config.reintroduce_with_median,
                            help='Reintroduce banks with the median of current banks')
        parser.add_argument('--seed', type=int, default=None, help='seed used for random generator')
        args, other_possible_config_args = parser.parse_known_args()
        if args.graph_stats:
            GraphStatistics.describe(args.graph_stats, interact=True)
        if args.t != model.config.T:
            model.config.T = args.t
        if args.n != model.config.N:
            model.config.N = args.n
        if args.eta != model.eta:
            model.eta = args.eta
        model.config.allow_replacement_of_bankrupted = not args.no_replace
        model.config.reintroduce_with_median = args.reintr_with_median
        model.config.psi_endogenous = args.psi_endogenous
        model.config.lender_change = determine_algorithm(args.lc)
        model.config.lender_change.set_parameter('m', args.lc_m)
        if not isinstance(model.config.lender_change,Preferential):
            model.config.lender_change.set_parameter('p', args.lc_p)
        model.config.define_values_from_args(other_possible_config_args)
        if model.config.seed and model.config.seed != model.default_seed and args.seed is None:
            args.seed = model.config.seed
        model.config.lender_change.set_initial_graph_file(args.lc_ini_graph_file)
        model.log.define_log(args.log, args.logfile, args.modules)
        model.statistics.define_output_format(args.output_format)
        model.statistics.set_gif_graph(args.gif_graph)
        model.statistics.define_plot_format(args.plot_format)
        model.statistics.define_detailed_equity(args.detailed_equity, args.save)
        Utils.run(args.save, Utils.__extract_t_values_from_arg__(args.graph),
                  output_directory=args.output, seed=args.seed,
                  interactive=args.log == 'ERROR' or args.logfile is not None)

    @staticmethod
    def run(save=None, save_graph_instants=None, interactive=False, output_directory=None, seed=None):
        global model
        if not save_graph_instants and Config.GRAPHS_MOMENTS:
            save_graph_instants = Config.GRAPHS_MOMENTS
        model.initialize(export_datafile=save, save_graphs_instants=save_graph_instants,
                         output_directory=output_directory, seed=seed)
        model.simulate_full(interactive=interactive)
        result = model.finish()
        if interactive and model.statistics.get_cross_correlation_result(0):
            print('\n'+model.statistics.get_cross_correlation_result(0))
            print(model.statistics.get_cross_correlation_result(1))
            print('bankruptcy.mean: %s' % model.statistics.bankruptcy.mean())
            print('interest_rate.mean: %s' % model.statistics.interest_rate.mean())
        return result

    @staticmethod
    def is_notebook():
        try:
            __IPYTHON__
            return get_ipython().__class__.__name__ != 'SpyderShell'
        except NameError:
            return False

    @staticmethod
    def is_spyder():
        try:
            return get_ipython().__class__.__name__ == 'SpyderShell'
        except NameError:
            return False
model = Model()
if Utils.is_notebook():
    model.statistics.OUTPUT_DIRECTORY = '/content'
    model.statistics.output_format = 'csv'
    Utils.run(save='results')
elif __name__ == '__main__':
    Utils.run_interactive()

<span style="color:red"><b>>>>>>>> remote</b></span>